In [1]:
# data downloading
import data_util
import pandas as pd
from collections import defaultdict
import os
import sys

os.chdir("/home/yxjiang/source/ml_playground")
print(os.getcwd())

from util import config

/home/yxjiang/source/ml_playground


In [6]:
dataset_url="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dest_dir = "/tmp/data"
dataset_folder_path = os.path.join(dest_dir, "aclImdb")
data_util.download_data(url=dataset_url, dest_dir=dest_dir)

# generate word to id mapping
word_to_id, word_list = data_util.get_vocabulary(folder_path=dataset_folder_path, file_suffix="vocab")
print("There size of vocabulary is :", len(word_to_id))

Destination folder [/tmp/data] exists.
Target file [aclImdb_v1.tar.gz] exists, skip downloading.
Start to extract [/tmp/data/aclImdb_v1.tar.gz] to [/tmp/data]...
File extracted
Processing vocabulary from [/tmp/data/aclImdb].
There size of vocabulary is : 89527


In [ ]:
# put everything together
import time
from data_util import *
from models import *
from trainer import classification_trainer
from util import config

import torch.optim as optim
import torch.nn.functional as F


def run(rnn_type, checkpoint_prefix, **kwargs):
    cfg = config.Config(
        rnn_type=rnn_type,
        criteria=nn.CrossEntropyLoss, optimizer=optim.Adam, lr=0.00002, epochs=200, 
        batch_size=128, num_classes=2, sentence_max_length=20, word_embedding_length=128, 
        activation=F.relu, dropout=0.1, **kwargs
    )

    pos_train_data_folder = os.path.join(dataset_folder_path, "train/pos")
    neg_train_data_folder = os.path.join(dataset_folder_path, "train/neg")
    train_dataset = MovieReviewDataset(cfg, pos_train_data_folder, neg_train_data_folder, word_to_id, 
                                transform=transforms.Compose([
                                    TruncateTransform(cfg), 
                                    WordsToIdsTransform(cfg, word_to_id),
                                ]))

    pos_test_data_folder = os.path.join(dataset_folder_path, "test/pos")
    neg_test_data_folder = os.path.join(dataset_folder_path, "test/neg")
    test_dataset = MovieReviewDataset(cfg, pos_test_data_folder, neg_test_data_folder, word_to_id, 
                                transform=transforms.Compose([
                                    TruncateTransform(cfg), 
                                    WordsToIdsTransform(cfg, word_to_id),
                                ]))

    train_dataloader = DataLoader(dataset=train_dataset, batch_size=cfg.batch_size, shuffle=True, collate_fn=data_util.pad_collate)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=cfg.batch_size, collate_fn=data_util.pad_collate)

    # model = TextCNN(cfg, len(word_to_id)).to(device)
    model = RNN(cfg, len(word_list)).to(cfg.device)

    classification_trainer.train(model=model, config=cfg, train_dataloader=train_dataloader, test_dataloader=test_dataloader, checkpoint_prefix=checkpoint_prefix, check_interval=10, kwargs=kwargs)

# run(rnn_type=nn.GRU, num_layers=3, num_directions=2)
# run(rnn_type=nn.GRU, num_layers=1, num_directions=1)
# run(nn.LSTM, num_layers=3, num_directions=2)
run(nn.LSTM, checkpoint_prefix='lstm', num_layers=1, num_directions=1, existing_checkpoint_filepath='/tmp/model/lstm_80.ckpt')

In [21]:
# Profiling model performance
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from data_util import *
from models import *
from util import config

import torch.autograd.profiler as profiler

cfg = config.Config(
    rnn_type=nn.LSTM,
    criteria=nn.CrossEntropyLoss, optimizer=optim.Adam, lr=0.00002, epochs=200, 
    batch_size=128, num_classes=2, sentence_max_length=20, word_embedding_length=128, 
    activation=F.relu, dropout=0.1, num_layers=1, num_directions=1,
)

# Load the model
checkpoint = torch.load('/tmp/model/lstm_80.ckpt')
model = RNN(cfg, len(word_list)).to(cfg.device)
model.load_state_dict(checkpoint['state_dict'])

# load the data
pos_train_data_folder = os.path.join(dataset_folder_path, "train/pos")
neg_train_data_folder = os.path.join(dataset_folder_path, "train/neg")
train_dataset = MovieReviewDataset(
    cfg, pos_train_data_folder, neg_train_data_folder, word_to_id, 
    transform=transforms.Compose([
        TruncateTransform(cfg), 
        WordsToIdsTransform(cfg, word_to_id),
    ]))
train_dataloader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True, collate_fn=data_util.pad_collate)


with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        word_ids, labels, x_lens, y_lens = next(iter(train_dataloader))
        model(word_ids.to(cfg.device), x_lens)

print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

prof.export_chrome_trace("trace.json")

------------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  -----------------------------------  
Name                                  Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     Number of Calls  Input Shapes                         
------------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  -----------------------------------  
model_inference                       62.16%           1.247ms          99.81%           2.003ms          2.003ms          1                []                                   
lstm                                  1.19%            23.936us         14.23%           285.396us        285.396us        1                [[20, 128], [20]]                    
_cudnn_rnn                            12.69%           254.574us        12.69%           254.574us        254